## Install invana-connector

In [20]:
# for stable version 
# pip install invana

# for installation specific branch
#pip install git+https://github.com/invana/invana@refactor/invana-connectors#egg=invana

# for latest code 
import sys
sys.path.append("../")

In [21]:
def print_list(list_data):
    for data in list_data:
        print("",data)

## 1. Modelling graph

### 1.1 Creating graph instance

In [22]:
from invana import InvanaGraph

gremlin_server_url = "ws://megamind.local:8182/gremlin"

graph = InvanaGraph(gremlin_server_url)

[DEBUG:2023-05-15 14:55:39,958:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTING
[DEBUG:2023-05-15 14:55:39,959:connector._init_connection:99] - create driver connection  
[DEBUG:2023-05-15 14:55:40,967:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTED


### 1.2 Defining Nodes and Relationship Models

In [23]:
from invana.ogm.models import VertexModel, EdgeModel
from invana.ogm.fields import StringProperty, DateTimeProperty, IntegerProperty, FloatProperty, BooleanProperty
from invana.ogm import indexes
from datetime import datetime


class Project(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(max_length=30, trim_whitespaces=True),
        'description': StringProperty(allow_null=True, min_length=5),
        'rating': FloatProperty(allow_null=True),
        'is_active': BooleanProperty(default=True),
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }
    indexes = (
        indexes.CompositeIndex("name"),
        indexes.CompositeIndex("created_at")
    )


class Person(VertexModel):
    graph = graph
    properties = {
        'first_name': StringProperty(min_length=5, trim_whitespaces=True),
        'last_name': StringProperty(allow_null=True),
        'username': StringProperty(default="rrmerugu"),
        'member_since': IntegerProperty(),

    }
    indexes = (
        indexes.CompositeIndex("username"),
    )


class Authored(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }
    indexes = (
        indexes.CompositeIndex("created_at")
    )

### 1.3 Installing models on graph

In [24]:
from invana.helpers.helpers import install_models

install_models = False

if install_models:
    # i_understand_rollback=True will roll back any transactions in pending.
    install_models(graph, Project, Person, Authored, i_understand_rollback=True)
else:
    print(f"Ignoring models installation because install_models={install_models}")

Ignoring models installation because install_models=False


## 2 CRUD Operations on data

### 2.1 Nodes

#### 2.1.1 Create Nodes 

##### 2.1.1.1 get_or_create

In [25]:
person_ravi = Person.objects.get_or_create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)
person_invana = Person.objects.get_or_create(first_name="Invana", last_name="Technologies", member_since=2016)

##### 2.1.1.2 create

In [26]:
project_iengine = Project.objects.create(name="invana engine", rating=5, is_active=True)
project_iconnector = Project.objects.create(name="invana connectors", rating=5, is_active=True)
project_istudio = Project.objects.create(name="invana studio", rating=5, is_active=True)
project_bengine = Project.objects.create(name="browser engine", rating=5, is_active=False)


#### 2.1.2 Read Nodes

##### 2.1.2.1 search 

In [27]:
# search
project_search = Project.objects.search().to_list()
print(project_search)

[<Node:Project id="8248" is_active=True name=invana connectors rating=5.0 created_at=2023-05-15 14:18:13.772000 >, <Node:Project id="4176" is_active=True name=invana engine rating=5.0 created_at=2023-05-15 14:18:13.665000 >, <Node:Project id="8272" is_active=False name=browser engine rating=5.0 created_at=2023-05-15 14:18:13.839000 >, <Node:Project id="12368" is_active=True name=invana studio rating=5.0 created_at=2023-05-15 14:56:01.214000 >, <Node:Project id="4304" is_active=True name=invana studio rating=5.0 created_at=2023-05-15 14:18:13.809000 >, <Node:Project id="8400" is_active=True name=invana engine rating=5.0 created_at=2023-05-15 14:55:51.105000 >, <Node:Project id="12496" is_active=False name=browser engine rating=5.0 created_at=2023-05-15 14:56:01.238000 >, <Node:Project id="4336" is_active=True name=invana connectors rating=5.0 created_at=2023-05-15 14:55:56.154000 >]


##### 2.1.2.3 search (by node id)

In [28]:
ids = [p.id for p in project_search] 

project_search_has__id = Project.objects.search(has__id=project_search[0].id).to_list()
project_search_has__id_within = Project.objects.search(has__id__within=ids[:2]).to_list()

print("project_search_has__id", project_search_has__id)
print("=======")
print("project_search_has__id_within", project_search_has__id_within)

project_search_has__id [<Node:Project id="8248" is_active=True name=invana connectors rating=5.0 created_at=2023-05-15 14:18:13.772000 >]
project_search_has__id_within [<Node:Project id="8248" is_active=True name=invana connectors rating=5.0 created_at=2023-05-15 14:18:13.772000 >, <Node:Project id="4176" is_active=True name=invana engine rating=5.0 created_at=2023-05-15 14:18:13.665000 >]


##### 2.1.2.2 search (startingWith)

In [29]:
import pprint
project_search_startsWith = Project.objects.search(has__name__startingWith="browser").to_list()
print(project_search_startsWith)

[<Node:Project id="8272" is_active=False name=browser engine rating=5.0 created_at=2023-05-15 14:18:13.839000 >, <Node:Project id="12496" is_active=False name=browser engine rating=5.0 created_at=2023-05-15 14:56:01.238000 >]


##### 2.1.2.3 Other search 

In [30]:

# search string - eq, neq, startingWith, endingWith, containing, notStartingWith, notEndingWith, notContaining
Project.objects.search(has__name__eq="invana").to_list()
Project.objects.search(has__name__neq="invana").to_list()
Project.objects.search(has__name__startingWith="invana").to_list()
Project.objects.search(has__name__endingWith="engine").to_list()
Project.objects.search(has__name__containing="eng").to_list()
Project.objects.search(has__name__notStartingWith="invana").to_list()
Project.objects.search(has__name__notEndingWith="engine").to_list()
Project.objects.search(has__name__notContaining="ravi").to_list()

# lt, gt, lte, gte, inside, outside, between
Person.objects.search(has__member_since__lte=3000).to_list()
Person.objects.search(has__member_since__lt=3000).to_list()
Person.objects.search(has__member_since__gte=1999).to_list()
Person.objects.search(has__member_since__gt=1999).to_list()


[<Node:Person id="4136" member_since=2000 last_name=Merugu first_name=Ravi Raja username=rrmerugu >,
 <Node:Person id="4152" member_since=2016 last_name=Technologies first_name=Invana username=rrmerugu >]

In [31]:
# inside_query = Person.objects.search(has__member_since=2000).to_list()

inside_query = Person.objects.search(has__member_since__inside=(1000, 3000)).to_list()
outside_quuery = Person.objects.search(has__member_since__outside=(1000, 2005)).to_list()
between_query = Person.objects.search(has__member_since__between=(1000, 3000)).to_list()
print("==inside_query", inside_query)
print("==outside_quuery", outside_quuery)
print("==between_query", between_query)

==inside_query [<Node:Person id="4136" member_since=2000 last_name=Merugu first_name=Ravi Raja username=rrmerugu >, <Node:Person id="4152" member_since=2016 last_name=Technologies first_name=Invana username=rrmerugu >]
==outside_quuery [<Node:Person id="4152" member_since=2016 last_name=Technologies first_name=Invana username=rrmerugu >]
==between_query [<Node:Person id="4136" member_since=2000 last_name=Merugu first_name=Ravi Raja username=rrmerugu >, <Node:Person id="4152" member_since=2016 last_name=Technologies first_name=Invana username=rrmerugu >]


##### 2.1.2.2 count

In [32]:
project_count = Project.objects.search().count()
print(project_count)

8


In [33]:
# Project.objects.delete()
# Person.objects.delete()
# Authored.objects.delete()